In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Data Processing

In [ ]:
import os
import cv2
import numpy as np

!pip install yt_dlp

import yt_dlp

import matplotlib.pyplot as plt

def download_youtube_video(url, output_path):
    ydl_opts = {
        'format': 'bestvideo[height=1080][ext=mp4]/mp4',
        'outtmpl': output_path,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return output_path

def video_to_3d_data_frame(video_path, segment_number):
    video_data = []
    start_frame = (segment_number - 1) * 3000
    end_frame = segment_number * 3000
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return None

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # move the current position to the start frame

    current_frame = start_frame
    while current_frame < end_frame and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        video_data.append(rgb_frame)
        current_frame += 1

    cap.release()
    video_data_array = np.array(video_data)
    return video_data_array

def yt_to_data(url, segment_number):
    youtube_url = url
    video_path = 'downloaded_video.mp4'
    downloaded_video_path = download_youtube_video(youtube_url, video_path)
    video_data_array = video_to_3d_data_frame(downloaded_video_path, segment_number)

    #os.remove(downloaded_video_path)  # ensure the file is removed after processing

    return video_data_array


train_url1 = 'https://www.youtube.com/watch?v=j0HoMaaQj9I'
train_url2 = 'https://www.youtube.com/watch?v=1jzJGcRdxPY'
train_url3 = 'https://www.youtube.com/watch?v=Bs58RoTf-g8'
train_url4 = 'https://www.youtube.com/watch?v=XoyYtqi5u54'
train_url5 = 'https://www.youtube.com/watch?v=rMPkUuMq024'
train_url6 = 'https://www.youtube.com/watch?v=JUWtWPX6hgs'
train_url7 = 'https://www.youtube.com/watch?v=WtnbT6ft710'
train_url8 = 'https://www.youtube.com/watch?v=QLF0FXcW25E'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


### Setting up Training Pairs for Data

Keep in mind how many iterations are necessary for a samples of 3000 frames per video. You can calculate the total number of frames by looking at the 'stats for nerds' setting on the YouTube video and multiplying the FPS by the length of the video. Then divide that by 3000 to determine the number of iterations necessary. Then add that to the urls dictionary

In [ ]:
import torch
from torch.nn import functional

def create_training_pairs(video_data_array):
    print("Starting the Numpy!")
    length = len(video_data_array) - 2
    # pre-allocate tensors
    input_frames = torch.zeros((length // 2, 2, *video_data_array.shape[1:]), dtype=video_data_array.dtype)
    target_frames = torch.zeros((length // 2, *video_data_array.shape[1:]), dtype=video_data_array.dtype)

    print("Entering loop")
    for i in range(0, length - 1, 2):
        input_frames[i // 2] = video_data_array[i:i+2]  # two consecutive frames
        target_frames[i // 2] = video_data_array[i+1]   # middle frame
        if i % 100 == 0:
          print(i, "/", length, flush=True)
    print("Finished loop")
    return input_frames, target_frames

urls = {train_url1 : 1,
        train_url2 : 1,
        train_url3 : 5,
        train_url4 : 2,
        train_url5 : 1,
        train_url6 : 6,
        train_url7 : 1,
        train_url8 : 11
        }

### Creating the hybrid Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
os.remove('downloaded_video.mp4')

FileNotFoundError: [Errno 2] No such file or directory: 'downloaded_video.mp4'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset

class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size):
        super(ConvLSTMCell, self).__init__()

        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2

        self.conv = nn.Conv2d(
            in_channels=self.input_channels + self.hidden_channels,
            out_channels=4 * self.hidden_channels,
            kernel_size=self.kernel_size,
            padding=self.padding,
            bias=True
        )

    def forward(self, x, h_prev, c_prev):
        combined = torch.cat([x, h_prev], dim=1)
        conv_output = self.conv(combined)

        i, f, o, g = torch.split(conv_output, self.hidden_channels, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c_next = f * c_prev + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

class ConvLSTM(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size, num_layers):
        super(ConvLSTM, self).__init__()

        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.num_layers = num_layers

        self.cell_list = nn.ModuleList()

        for i in range(self.num_layers):
            cur_input_channels = self.input_channels if i == 0 else self.hidden_channels
            self.cell_list.append(ConvLSTMCell(cur_input_channels, self.hidden_channels, self.kernel_size))

    def forward(self, x):
        batch_size, seq_len, _, height, width = x.size()

        h = [torch.zeros(batch_size, self.hidden_channels, height, width, device=x.device)
             for _ in range(self.num_layers)]
        c = [torch.zeros(batch_size, self.hidden_channels, height, width, device=x.device)
             for _ in range(self.num_layers)]

        output = []

        for t in range(seq_len):
            x_t = x[:, t, :, :, :]

            for layer_idx in range(self.num_layers):
                h[layer_idx], c[layer_idx] = self.cell_list[layer_idx](x_t, h[layer_idx], c[layer_idx])
                x_t = h[layer_idx]

            output.append(h[-1])

        return torch.stack(output, dim=1)

class FrameInterpolationConvLSTM(nn.Module):
    def __init__(self):
        super(FrameInterpolationConvLSTM, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )

        # ConvLSTM
        self.convlstm = ConvLSTM(input_channels=256, hidden_channels=256, kernel_size=3, num_layers=2)

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
        self.lstm_activation = nn.ReLU()

    def forward(self, frame1, frame2):
        # Encode both frames
        enc1 = self.encoder(frame1)
        enc2 = self.encoder(frame2)

        # Combine encoded frames for ConvLSTM input
        conv_lstm_input = torch.stack([enc1, enc2], dim=1)  # Shape: [batch_size, 2, channels, height, width]

        # Pass through ConvLSTM
        conv_lstm_output = self.convlstm(conv_lstm_input)

        conv_lstm_output = self.lstm_activation(conv_lstm_output)

        # Take the last output (interpolated frame features)
        interpolated_features = conv_lstm_output[:, -1]

        # Decode to get the final interpolated frame
        output = self.decoder(interpolated_features)

        return output

class FrameDataset(Dataset):
    def __init__(self, input_frames, target_frames):
        self.input_frames = input_frames
        self.target_frames = target_frames

    def __len__(self):
        return len(self.target_frames)

    def __getitem__(self, idx):
        # normalize pixel data to range [0, 1]
        frame1 = self.input_frames[idx][0].float() / 255.0
        frame2 = self.input_frames[idx][1].float() / 255.0
        target_frame = self.target_frames[idx].float() / 255.0

        # permute the dimensions to [C, H, W]
        frame1 = frame1.permute(2, 0, 1)
        frame2 = frame2.permute(2, 0, 1)
        target_frame = target_frame.permute(2, 0, 1)

        return (frame1, frame2), target_frame


model = FrameInterpolationConvLSTM().to(device)

#from google.colab import files
#from google.colab import drive
#drive.mount('/content/drive')
#path = "/content/drive/My Drive/model_hybrid_delta1_9epoch.pth"

#state_dict = torch.load(path, map_location=device)
#model.load_state_dict(state_dict)
#model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 1

train_array = None
input_frames, target_frames = None, None

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    counter = 1
    for url in urls:
      for j in range(urls[url]):

        train_array = yt_to_data(url, j+1)
        train_array = torch.from_numpy(train_array)
        input_frames, target_frames = create_training_pairs(train_array)
        train_array = None
        frame_dataset = FrameDataset(input_frames, target_frames)
        train_loader = DataLoader(frame_dataset, batch_size=10, shuffle=False, num_workers=4) # I ran it on 12 CPU Cores, use GPU if you can

        for i, ((frame1, frame2), target_frame) in enumerate(train_loader):
          frame1, frame2, target_frame = frame1.to(device), frame2.to(device), target_frame.to(device)

          # Zero the parameter gradients
          optimizer.zero_grad()

          # Forward pass
          outputs = model(frame1, frame2)

          # Compute loss
          loss = criterion(outputs, target_frame)

          # Backward pass and optimize
          loss.backward()
          optimizer.step()

          # print statistics so that we can tell the progress of this model as it runs
          running_loss += loss.item()
          if i % 10 == 9:    # print every 10 mini-batches
              print(f'Url: {counter}, Sample: {j+1}, Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 10:.4f}')
              running_loss = 0.0

        input_frames, target_frames = None, None
        frame_dataset = None
        train_loader = None
      os.remove('downloaded_video.mp4')
      counter += 1

print('Finished Training')


Mounted at /content/drive
[youtube] Extracting URL: https://www.youtube.com/watch?v=j0HoMaaQj9I
[youtube] j0HoMaaQj9I: Downloading webpage
[youtube] j0HoMaaQj9I: Downloading ios player API JSON
[youtube] j0HoMaaQj9I: Downloading player 8d9f6215
[youtube] j0HoMaaQj9I: Downloading m3u8 information
[info] j0HoMaaQj9I: Downloading 1 format(s): 614
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 16
[download] Destination: downloaded_video.mp4
[download] 100% of   14.99MiB in 00:00:01 at 14.09MiB/s                
Starting the Numpy!
Entering loop
0 / 2011
100 / 2011
200 / 2011
300 / 2011
400 / 2011
500 / 2011
600 / 2011
700 / 2011
800 / 2011
900 / 2011
1000 / 2011
1100 / 2011
1200 / 2011
1300 / 2011
1400 / 2011
1500 / 2011
1600 / 2011
1700 / 2011
1800 / 2011
1900 / 2011
2000 / 2011
Finished loop
Url: 1, Sample: 1, Epoch: 1, Batch: 10, Loss: 0.0007
Url: 1, Sample: 1, Epoch: 1, Batch: 20, Loss: 0.0003
Url: 1, Sample: 1, Epoch: 1, Batch: 30, Loss: 0.0002
Url: 1, Sample: 1, E

### Testing the Model

Lets now save the model to another device, just so that we may run it again to other devices

In [ ]:
device = "cuda"
model = model.to(device)

In [ ]:
torch.save(model.state_dict(), 'model_hybrid_delta2_10epoch.pth')

from google.colab import files

from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/model_hybrid_delta2_10epoch.pth"
torch.save(model.state_dict(), path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
